In [2]:
import simdna
from simdna import synthetic as sn
import numpy as np
loaded_motifs = sn.LoadedEncodeMotifs(simdna.ENCODE_MOTIFS_PATH, pseudocountProb=0.001)

In [3]:
#pwm_names_full = (["IRF_disc"+str(x+1) for x in range(6)]+["IRF_known"+str(x+1) for x in range(21)]
#                  +["SPI1_disc"+str(x+1) for x in range(3)]+["SPI1_known"+str(x+1) for x in range(4)])
#pwm_names_minimal = ["IRF_known9", "SPI1_known2"]

pwm_names_full = (["GATA_disc"+str(x+1) for x in range(6)]+["GATA_known"+str(x+1) for x in range(22)]
                  +["TAL1_disc"+str(x+1) for x in range(2)]+["TAL1_known"+str(x+1) for x in range(5)])
pwm_names_minimal = ["GATA_disc1", "TAL1_known1"]

for pwm_names, weights_file in [(pwm_names_full, "weights_all"),(pwm_names_minimal, "weights_minimal")]:

    log_odds_arrays = []

    for pwm_name in pwm_names:
        motif = loaded_motifs.getPwm(pwm_name).getRows()/(np.array([0.3, 0.2, 0.2, 0.3])[None,:])
        log_odds = np.log(np.array(motif))[::-1,:]
        log_odds_arrays.append(log_odds)

    longest = max(x.shape[0] for x in log_odds_arrays)
    same_size_log_odds_arrays = []
    for array in log_odds_arrays:
        full_array = np.zeros((longest, 4))
        total_pad = longest - array.shape[0]
        left_pad = int(total_pad/2)
        right_pad = total_pad - left_pad
        full_array[left_pad:(longest-right_pad),:] = array
        same_size_log_odds_arrays.append(full_array)

    np.save(weights_file+".npy",np.transpose(same_size_log_odds_arrays,(1,2,0))[:,None,:,:])

In [4]:
print(np.load("weights_all.npy").shape)
print(np.load("weights_minimal.npy").shape)

(22, 1, 4, 35)
(16, 1, 4, 2)


In [31]:
import keras
from keras.layers.convolutional import Convolution1DFromWeightFile

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5005)


In [32]:
Convolution1DFromWeightFile("weights.npy")

In [52]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution1DFromWeightFile
from keras.layers.pooling import MaxPoolingFilter1D
from keras.layers.core import Activation

model = Sequential()
model.add(Convolution1DFromWeightFile("weights.npy", input_shape=(200,4)))
model.add(Activation("relu"))
model.add(MaxPoolingFilter1D(pool_length=10))
model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy')

In [55]:
import simdna
data = simdna.synthetic.core.read_simdata_file("variableSpacingGrammarSimulation_motif1-GATA_disc1_motif2-TAL1_known1_seqLength200_numSeq1000.simdata.gz")

In [58]:
from avutils import util

onehot_data = np.array([util.seq_to_one_hot(x) for x in data.sequences])

In [62]:
print(data.embeddings[0][2])

pos-71_GATA_disc1-CCCGATAAGG-Gap7-TAL1_known1-CCAGCCAGATGGTGGC
